In [1]:
import sys
sys.path.append("..")
sys.path.append("../src/")
sys.path.append("../src/algs/")

from src import interface
from src import simulator
from src import algs

import numpy as np
import matplotlib.pyplot as plt

# Experiment 1: Greedy Vs Deferred Vs Lookahead
-- batch saved for later

In [2]:
def weight_function_distance(buyer_pos, buyer_d, seller_pos, seller_d):
    bx, by = buyer_pos
    sx, sy = seller_pos
    return 1/(1 + (bx-sx)**2 + (by-sy)**2)

In [9]:
departure_distr = (6,1) # mean, var
seed = 42
p_node = 1 # add a node every step
num_runs = 1000

In [10]:
algs_to_test = [
    ("greedy", algs.Greedy()),
    ("dfa", algs.DynamicDeferredAcceptance()),
    ("lookahead1", algs.DeferredWithLookAhead(1)),
    ("lookahead3", algs.DeferredWithLookAhead(3)),
    ("lookahead5", algs.DeferredWithLookAhead(5))
]

## actually run one of the experiments

In [11]:
results = {}
for (alg_name, alg) in algs_to_test:
    inter = interface.Interface(
        alg,
        simulator.Simulator(weight_function_distance),
        p_node=p_node,
        seed=seed,
        dep_distr=departure_distr,
        verbose=False)
    results[alg_name] = inter.run(num_runs)

In [12]:
for alg_name, res in results.items():
    average_weight =  np.mean([w[0] for w in res[0]])
    print("The {} algorithm had an average weight of {}".format(alg_name, average_weight))


The greedy algorithm had an average weight of 0.20547522907237778
The dfa algorithm had an average weight of 0.1729508515809003
The lookahead1 algorithm had an average weight of 0.1729508515809003
The lookahead3 algorithm had an average weight of 0.1729508515809003
The lookahead5 algorithm had an average weight of 0.1729508515809003


# Let's run a few experiments...

In [2]:
def run_an_experiment(seed, p_node, departure_distr, algs_to_test, weight_fn,
                      fewest_nodes_per_add=1, max_nodes_per_add=10,
                      p_buyer=0.5, num_runs=1000, size=(10,10)):
    results = {}
    for (alg_name, alg) in algs_to_test:
        inter = interface.Interface(
            alg,
            simulator.Simulator(weight_fn),
            p_node=p_node,
            min_to_add=fewest_nodes_per_add,
            max_to_add=max_nodes_per_add,
            p_buyer=p_buyer,
            seed=seed,
            dep_distr=departure_distr)
        results[alg_name] = inter.run(num_runs)
    for alg_name, res in results.items():
        average_weight =  np.mean([w[0] for w in res[0]])
        print("The {} algorithm had an average weight of {}".format(alg_name, average_weight))
    return results


In [3]:
def weight_function_distance(buyer_pos, buyer_d, seller_pos, seller_d):
    bx, by = buyer_pos
    sx, sy = seller_pos
    return 1/(1 + (bx-sx)**2 + (by-sy)**2)

In [7]:
algs_to_test = [
    ("greedy", algs.Greedy()),
    ("dfa", algs.DynamicDeferredAcceptance()),
    ("lookahead1", algs.DeferredWithLookAhead(1)),
    ("lookahead3", algs.DeferredWithLookAhead(3)),
    ("lookahead5", algs.DeferredWithLookAhead(5))
]
results1 = {}
for seed in range(0,1):
    print("\nSeed:", seed)
    results1[seed] = run_an_experiment(seed, 0.9, (6,1), algs_to_test, weight_function_distance,
                      fewest_nodes_per_add=1, max_nodes_per_add=6,
                      p_buyer=0.5, num_runs=500, size=(10,10))


Seed: 0
The greedy algorithm had an average weight of 0.43762006757505756
The dfa algorithm had an average weight of 0.3367995277769981
The lookahead1 algorithm had an average weight of 0.33395370981184247
The lookahead3 algorithm had an average weight of 0.35173405685815085
The lookahead5 algorithm had an average weight of 0.335085892787785


In [8]:
def weight_function_d_and_time(buyer_pos, buyer_d, seller_pos, seller_d):
    bx, by = buyer_pos
    sx, sy = seller_pos
    return (buyer_d + seller_d)/(1 + (bx-sx)**2 + (by-sy)**2)

In [10]:
results2 = {}
for seed in range(0,3):
    print("\nSeed:", seed)
    results2[seed] = run_an_experiment(seed, 0.9, (6,1), algs_to_test, 
                                      weight_function_d_and_time,
                      fewest_nodes_per_add=1, max_nodes_per_add=6,
                      p_buyer=0.5, num_runs=500, size=(10,10))


Seed: 0
The greedy algorithm had an average weight of 4.190568570766376
The dfa algorithm had an average weight of 3.4094300005207576
The lookahead1 algorithm had an average weight of 3.391157647947678
The lookahead3 algorithm had an average weight of 3.4407008314973893
The lookahead5 algorithm had an average weight of 3.2623433038262686

Seed: 1
The greedy algorithm had an average weight of 4.502464174251896
The dfa algorithm had an average weight of 3.577685942441412
The lookahead1 algorithm had an average weight of 3.577685942441412
The lookahead3 algorithm had an average weight of 3.577685942441412
The lookahead5 algorithm had an average weight of 3.577685942441412

Seed: 2
The greedy algorithm had an average weight of 4.034617277944727
The dfa algorithm had an average weight of 3.159626923177907
The lookahead1 algorithm had an average weight of 3.159626923177907
The lookahead3 algorithm had an average weight of 3.159626923177907
The lookahead5 algorithm had an average weight of 3

In [11]:
results3 = {}
for seed in range(0,1):
    print("\nSeed:", seed)
    results3[seed] = run_an_experiment(seed, 0.9, (6,1), algs_to_test, 
                                       weight_function_distance,
                      fewest_nodes_per_add=1, max_nodes_per_add=6,
                      p_buyer=0.6, num_runs=500, size=(10,10))


Seed: 0
The greedy algorithm had an average weight of 0.38988696186011984
The dfa algorithm had an average weight of 0.3519551667240272
The lookahead1 algorithm had an average weight of 0.34036242197733807
The lookahead3 algorithm had an average weight of 0.3383919733327125
The lookahead5 algorithm had an average weight of 0.3316427858889616


In [12]:
results3 = {}
for seed in range(0,1):
    print("\nSeed:", seed)
    results3[seed] = run_an_experiment(seed, 0.9, (6,1), algs_to_test, 
                                       weight_function_distance,
                      fewest_nodes_per_add=1, max_nodes_per_add=6,
                      p_buyer=0.4, num_runs=500, size=(10,10))


Seed: 0
The greedy algorithm had an average weight of 0.448084758378876
The dfa algorithm had an average weight of 0.39094075690305613
The lookahead1 algorithm had an average weight of 0.3692056478042986
The lookahead3 algorithm had an average weight of 0.3673403304161357
The lookahead5 algorithm had an average weight of 0.402344660879187


In [17]:
def print_some_things(result_dict):
    for seed in result_dict.keys():
        print("Seed:", seed)
        for alg_name, res in result_dict[seed].items():
            print("Alg:", alg_name)
            weights, discarded_buyers, discarded_sellers = res
            avg_weight_matching = np.mean([w[0] for w in weights])
            n_matched_nodes = len(weights)*2 # because a match is 2 nodes
            num_discarded_buyers = np.sum([d[0] for d in discarded_buyers])
            num_discarded_sellers = np.sum([d[0] for d in discarded_sellers])
            n_nodes = n_matched_nodes + num_discarded_buyers + num_discarded_sellers
            avg_weight_per_node = (avg_weight_matching * n_matched_nodes) / n_nodes
            print("Average weight per matching:", avg_weight_matching)
            print("Average weight per node:", avg_weight_per_node)
            print("Number of nodes: {}, matches: {}".format(n_nodes, len(weights)))
            print("Number of dropped buyers: {}, sellers: {}".format(
                num_discarded_buyers,
                num_discarded_sellers
            ))

In [18]:
print_some_things(results1)

Seed: 0
Alg: greedy
Average weight per matching: 0.43762006757505756
Average weight per node: 0.22143925982769933
Number of nodes: 1498, matches: 379
Number of dropped buyers: 368, sellers: 372
Alg: dfa
Average weight per matching: 0.3367995277769981
Average weight per node: 0.22870821927556487
Number of nodes: 1452, matches: 493
Number of dropped buyers: 249, sellers: 217
Alg: lookahead1
Average weight per matching: 0.33395370981184247
Average weight per node: 0.22491691111644582
Number of nodes: 1464, matches: 493
Number of dropped buyers: 274, sellers: 204
Alg: lookahead3
Average weight per matching: 0.35173405685815085
Average weight per node: 0.23305078613914287
Number of nodes: 1467, matches: 486
Number of dropped buyers: 241, sellers: 254
Alg: lookahead5
Average weight per matching: 0.335085892787785
Average weight per node: 0.22339059519185667
Number of nodes: 1470, matches: 490
Number of dropped buyers: 242, sellers: 248
